In [278]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [279]:
# TODO: create seperate a file for preprocessed data??
df = pd.read_csv('../data/raw/average-monthly-surface-temperature.csv')
df.reset_index()
df = df.rename(columns={'Average surface temperature': 'Daily Average Temp', 'Average surface temperature.1': 'Monthly Average Temp'})

In [280]:
df.sample(5)

,Entity,Code,year,Day,Daily Average Temp,Monthly Average Temp
23405,Bosnia and Herzegovina,BIH,2020,2020-06-15,17.463856,11.114944
191373,Uzbekistan,UZB,1992,1992-10-15,10.752309,12.438180
154795,Sao Tome and Principe,STP,2004,2004-08-15,23.834442,25.531410
190950,Uzbekistan,UZB,1957,1957-07-15,26.053324,11.868112
117365,Mexico,MEX,1945,1945-06-15,24.480135,19.915320


In [281]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198900 entries, 0 to 198899
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Entity                198900 non-null  object 
 1   Code                  198900 non-null  object 
 2   year                  198900 non-null  int64  
 3   Day                   198900 non-null  object 
 4   Daily Average Temp    198900 non-null  float64
 5   Monthly Average Temp  198900 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.1+ MB


In [282]:
df.describe()

,year,Daily Average Temp,Monthly Average Temp
count,198900.00000,198900.000000,198900.000000
mean,1982.00000,18.072073,18.072073
std,24.53575,10.246142,8.710114
min,1940.00000,-36.240032,-21.529121
25%,1961.00000,12.304079,10.569263
50%,1982.00000,22.055794,21.856285
75%,2003.00000,25.317015,25.142885
max,2024.00000,39.889374,29.794220


# Missing values

In [283]:
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# there are no missing values

Missing Values:
Entity                  0
Code                    0
year                    0
Day                     0
Daily Average Temp      0
Monthly Average Temp    0
dtype: int64


# Duplicates

In [284]:
duplicate_rows = df[df.duplicated()]
print("Duplicate Rows:")
print(duplicate_rows)

# there are no duplicate rows

Duplicate Rows:
Empty DataFrame
Columns: [Entity, Code, year, Day, Daily Average Temp, Monthly Average Temp]
Index: []


# Convert Data Types

In [285]:
df["Month"] = pd.to_datetime(df["Day"]).dt.month
df["Day"] = pd.to_datetime(df["Day"]).dt.day
df['Year'] = pd.to_datetime(df['year']).dt.year

df.drop(columns=["year"], axis=1, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198900 entries, 0 to 198899
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Entity                198900 non-null  object 
 1   Code                  198900 non-null  object 
 2   Day                   198900 non-null  int32  
 3   Daily Average Temp    198900 non-null  float64
 4   Monthly Average Temp  198900 non-null  float64
 5   Month                 198900 non-null  int32  
 6   Year                  198900 non-null  int32  
dtypes: float64(2), int32(3), object(2)
memory usage: 8.3+ MB


In [286]:
df

,Entity,Code,Day,Daily Average Temp,Monthly Average Temp,Month,Year
0,Afghanistan,AFG,15,-2.032494,11.327695,1,1970
1,Afghanistan,AFG,15,-0.733503,11.327695,2,1970
2,Afghanistan,AFG,15,1.999134,11.327695,3,1970
3,Afghanistan,AFG,15,10.199754,11.327695,4,1970
4,Afghanistan,AFG,15,17.942135,11.327695,5,1970
...,...,...,...,...,...,...,...
198895,Zimbabwe,ZWE,15,20.559408,22.921250,8,1970
198896,Zimbabwe,ZWE,15,23.642931,22.921250,9,1970
198897,Zimbabwe,ZWE,15,24.407030,22.921250,10,1970
198898,Zimbabwe,ZWE,15,25.672321,22.921250,11,1970


# Verifying Entity and Code columns

In [287]:
for name, num_of_occurrences in df['Entity'].value_counts().items():
    # check if every country has the same number of entries
    if num_of_occurrences != 1020:
        print(name, "has", num_of_occurrences)

for name in df["Entity"].unique():
    # check if every country name (Entity column) matches the country code 
    correct_code = df.loc[df["Entity"] == name].iloc[0]["Code"]      
    incorrect_rows = df.loc[(df["Entity"] == name) & (df["Code"] != correct_code)]
    if incorrect_rows.shape[0] > 0:
        display(incorrect_rows)


# Drop Unnecessary Columns

In [ ]:
# having both "Enity" and "Code" is redundant, so I'm dropping a column
df.drop("Code", axis=1, inplace=True, errors='ignore')
df

,Entity,Day,Daily Average Temp,Monthly Average Temp,Month,Year
0,Afghanistan,15,-2.032494,11.327695,1,1970
1,Afghanistan,15,-0.733503,11.327695,2,1970
2,Afghanistan,15,1.999134,11.327695,3,1970
3,Afghanistan,15,10.199754,11.327695,4,1970
4,Afghanistan,15,17.942135,11.327695,5,1970
...,...,...,...,...,...,...
198895,Zimbabwe,15,20.559408,22.921250,8,1970
198896,Zimbabwe,15,23.642931,22.921250,9,1970
198897,Zimbabwe,15,24.407030,22.921250,10,1970
198898,Zimbabwe,15,25.672321,22.921250,11,1970


# Encode Code Column
Use One-Hot-Encoding

In [289]:
from sklearn.preprocessing import OneHotEncoder

In [290]:
df['Entity'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas',
       'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire",
       'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       'Democratic Republic of Congo', 'Denmark', 'Djibouti',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Falkland Islands', 'Faroe Islands',
       'Fiji', 'Finland', 'France', 'French Polynesia', 'Gabon', 

In [291]:
ohe = OneHotEncoder(sparse_output=False).set_output(transform='pandas')
ohe_transform = ohe.fit_transform(df[["Entity"]])
encoded_df = df.drop('Entity', axis=1) 
encoded_df = pd.concat([encoded_df, ohe_transform], axis=1)
encoded_df

,Day,Daily Average Temp,Monthly Average Temp,Month,Year,Entity_Afghanistan,Entity_Albania,Entity_Algeria,Entity_American Samoa,Entity_Andorra,...,Entity_United States Virgin Islands,Entity_Uruguay,Entity_Uzbekistan,Entity_Vanuatu,Entity_Venezuela,Entity_Vietnam,Entity_World,Entity_Yemen,Entity_Zambia,Entity_Zimbabwe
0,15,-2.032494,11.327695,1,1970,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15,-0.733503,11.327695,2,1970,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15,1.999134,11.327695,3,1970,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,10.199754,11.327695,4,1970,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15,17.942135,11.327695,5,1970,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198895,15,20.559408,22.921250,8,1970,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
198896,15,23.642931,22.921250,9,1970,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
198897,15,24.407030,22.921250,10,1970,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
198898,15,25.672321,22.921250,11,1970,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Data Visualization

In [292]:
'''
Global Temperature Trend Over Time
Overall trend of average surface temperatures from 1940 to 2024
'''

# For each year, average the 

'\nGlobal Temperature Trend Over Time\nOverall trend of average surface temperatures from 1940 to 2024\n'

In [293]:
'''
# plot average temperatures for each year
avg_temps_by_year = df.groupby("year").agg(
    avg_temp=("Daily Average Temp", "mean"),
).reset_index()

avg_temps_by_year = pd.melt(avg_temps_by_year, ["year"], value_name='Temperature (°C)', var_name="Legend")
sns.lineplot(data=avg_temps_by_year, x="year", y='Temperature (°C)', hue="Legend")
'''

'\n# plot average temperatures for each year\navg_temps_by_year = df.groupby("year").agg(\n    avg_temp=("Daily Average Temp", "mean"),\n).reset_index()\n\navg_temps_by_year = pd.melt(avg_temps_by_year, ["year"], value_name=\'Temperature (°C)\', var_name="Legend")\nsns.lineplot(data=avg_temps_by_year, x="year", y=\'Temperature (°C)\', hue="Legend")\n'

In [294]:
'''
# plot average temperatures for each month (Day column)
df['Month'] = pd.to_datetime(df['Day']).dt.month
#df.reset_index()
avg_temps_by_month = df.groupby(["Month", "year"]).aggregate(
    avg_temp=("Monthly Average Temp", "mean"),
    countries=("Code", "count"),
)

#avg_temps_by_month = pd.melt(avg_temps_by_month, ["Month"], value_name='Temperature (°C)', var_name="Legend")
#sns.lineplot(data=avg_temps_by_month, x="Month", y='Temperature (°C)', hue="Legend")
#df.sample(10)

#print(avg_temps_by_month.first())
#avg_temps_by_month
#avg_temps_by_month = df.groupby(by="Month").sum()
display(avg_temps_by_month)

# TODO: do bar chart for this
'''

'\n# plot average temperatures for each month (Day column)\ndf[\'Month\'] = pd.to_datetime(df[\'Day\']).dt.month\n#df.reset_index()\navg_temps_by_month = df.groupby(["Month", "year"]).aggregate(\n    avg_temp=("Monthly Average Temp", "mean"),\n    countries=("Code", "count"),\n)\n\n#avg_temps_by_month = pd.melt(avg_temps_by_month, ["Month"], value_name=\'Temperature (°C)\', var_name="Legend")\n#sns.lineplot(data=avg_temps_by_month, x="Month", y=\'Temperature (°C)\', hue="Legend")\n#df.sample(10)\n\n#print(avg_temps_by_month.first())\n#avg_temps_by_month\n#avg_temps_by_month = df.groupby(by="Month").sum()\ndisplay(avg_temps_by_month)\n\n# TODO: do bar chart for this\n'

In [295]:
'''
new_df = pd.DataFrame()
new_df['Months'] = df['Month']
'''

"\nnew_df = pd.DataFrame()\nnew_df['Months'] = df['Month']\n"

In [296]:
#df.drop("Month", axis=1)

# Export preprocessed dataset to new CSV file

In [297]:
encoded_df.to_csv('../data/processed/preprocessed_data.csv', index=False)